# Задание 1. Базовое изучение

Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/viktoria/Documents/Netology/theory/10_Statistics_basics/for_HW/horse_data.csv', na_values = '?', header=None)

In [3]:
df_horse = df.iloc[:, [0,1,3,4,11,12,19,22]]
df_horse.columns = ['surgery','age','rectal temperature','pulse','peristalsis','abdominal distension','total protein','outcome']
df_horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   surgery               299 non-null    float64
 1   age                   300 non-null    int64  
 2   rectal temperature    240 non-null    float64
 3   pulse                 276 non-null    float64
 4   peristalsis           256 non-null    float64
 5   abdominal distension  244 non-null    float64
 6   total protein         267 non-null    float64
 7   outcome               299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [5]:
df_horse[['surgery','age']].value_counts()
#представленный набор данных содержит информацию о 275 взрослых лошадях, большая часть из которых (162) была прооперирована; возраст 24 особей не определен.

surgery  age
1.0      1      162
2.0      1      113
1.0      9       18
2.0      9        6
dtype: int64

In [6]:
df_horse['rectal temperature'].describe()
#данные по ректальной температуре есть для 240 лошадей из 300; минимальное и максимальное значения температур - релевантны, т.е. выбросы отсутствуют

count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64

In [7]:
df_horse['pulse'].describe()
#данные по пульсу есть для 276 лошадей из 300
#нормальное значение пульса - 30-40 ударов в минуту; в научных статьях сообщается: "частота сердечных сокращений более 60 ударов в минуту указывает на тяжелое состояние и должна рассматриваться как чрезвычайная ситуация", НО медианное значение в 64.0 свидетельствует о том, что значения больше 60 не выбросы

count    276.000000
mean      71.913043
std       28.630557
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

In [8]:
df_horse['total protein'].describe()
#данные по содержанию белка есть для 267 лошадей из 300; нормальные значения лежат в диапазоне 6-7.5 (gms/dL), следовательно, допущены ошибки при вводе некоторых данных

count    267.000000
mean      24.456929
std       27.475009
min        3.300000
25%        6.500000
50%        7.500000
75%       57.000000
max       89.000000
Name: total protein, dtype: float64

In [ ]:
#для столбцов 'peristalsis', 'abdominal distension', 'outcome' данные отсутствуют для 44, 56 и 1 лошадей соответственно

# Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [71]:
#выбросы в сформированном наборе данных содержатся в стобце 'total protein', гипотезу подтверждает значение std - 27.47

In [9]:
#данные столбца 'total protein' позволяют выдвинуть гипотезу о пропуске десятичного разделителя в значениях, представленных двузначными числами
#принято решение скорректировать данные, разделив двузначные числа
df_horse_final = df_horse.copy()

def return_comma(protein):
    if protein >= 9.9:
        protein = protein / 10
    return protein

df_horse_final['total protein'] = df_horse_final['total protein'].apply(return_comma)

In [10]:
#стоит отметить, что значение "9" в столбце "age" не соответствует описанию набора данных
df_horse.sort_values(by=['age'], ascending=False).head(30)
#при распределении по зачению 9 в столбце "age" видим, что значения пульса у данных особей - около или более 100 ударов в минуту, а высокий пульс характерен для молодых особей
#можно предположить, что вместо 9 должно быть 2 = Young (< 6 months)
#принято решение скорректировать данные, заменив 9 на 2
def change_value(age):
    if age == 9:
        age = 2
    return age

df_horse_final['age'] = df_horse_final['age'].apply(change_value)

# Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [11]:
(df_horse.isna().mean() * 100).round(2)

surgery                  0.33
age                      0.00
rectal temperature      20.00
pulse                    8.00
peristalsis             14.67
abdominal distension    18.67
total protein           11.00
outcome                  0.33
dtype: float64

In [12]:
#принято решение пропуски в столбцах "surgery" и "outcome" заполнить модой, т.к. в каждом столбце отсутствует всего 1 значение, и корреляцию с другими показателями провести не удалось

df_horse_final['surgery'].fillna(df_horse_final['surgery'].mode()[0], inplace=True)
df_horse_final['outcome'].fillna(df_horse_final['outcome'].mode()[0], inplace=True)

In [13]:
df_horse_final[['outcome','peristalsis']].value_counts()

#было выявлено различное значение моды для столбца 'peristalsis' при группировке по столбцу 'outcome'
#принято решение пропуски в столбце 'peristalsis' заполнить модой, ориентируясь на значения столбца 'outcome'

df_horse_final['peristalsis'].fillna(df_horse_final.groupby(['outcome'])['peristalsis'].transform(lambda x: x.value_counts().idxmax()), inplace=True)

In [14]:
df_horse_final.groupby(['peristalsis'])['rectal temperature','pulse','total protein'].median()

#было выявлено различные значения медианы для столбцов 'rectal temperature', 'pulse' и 'total protein' при сортировке по столбцу 'peristalsis'
#принято решение пропуски в столбцах 'rectal temperature', 'pulse' и 'total protein' заполнить медианами, ориентируясь на значения столбца 'peristalsis'

df_horse_final['rectal temperature'].fillna(df_horse_final.groupby(['peristalsis'])['rectal temperature'].transform('median'), inplace=True)
df_horse_final['pulse'].fillna(df_horse_final.groupby(['peristalsis'])['pulse'].transform('median'), inplace=True)
df_horse_final['total protein'].fillna(df_horse_final.groupby(['peristalsis'])['total protein'].transform('median'), inplace=True)

<ipython-input-14-e22a2700898c>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_horse_final.groupby(['peristalsis'])['rectal temperature','pulse','total protein'].median()


In [24]:
df_horse_final[['peristalsis','outcome','abdominal distension']].value_counts()

#принято решение пропуски в столбце 'abdominal distension' заполнить модой, ориентируясь на значения столбцов 'peristalsis' и 'outcome', содержащих больше категориальных значений

df_horse_final['abdominal distension'].fillna(df_horse_final.groupby(['peristalsis','outcome'])['abdominal distension'].transform(lambda x: x.value_counts().idxmax()), inplace=True)

In [25]:
(df_horse_final.isna().mean() * 100).round(2)

surgery                 0.0
age                     0.0
rectal temperature      0.0
pulse                   0.0
peristalsis             0.0
abdominal distension    0.0
total protein           0.0
outcome                 0.0
dtype: float64

In [30]:
df_horse_final.head() #датафрейм, в котором отсутствуют пропуски

,surgery,age,rectal temperature,pulse,peristalsis,abdominal distension,total protein,outcome
0,2.0,1,38.5,66.0,4.0,4.0,8.4,2.0
1,1.0,1,39.2,88.0,4.0,2.0,8.5,3.0
2,2.0,1,38.3,40.0,3.0,1.0,6.7,1.0
3,1.0,2,39.1,164.0,4.0,4.0,7.2,2.0
4,2.0,1,37.3,104.0,3.0,3.0,7.4,2.0
